# Especializar um Modelo Pré-treinado do Hugging Face

Este notebook apresenta um roteiro que detalha os passos necessários para utilizar e especializar modelos de linguagem pré-treinados para a tarefa de sumarização de textos em português, garantindo a instalação e configuração adequadas das bibliotecas necessárias, bem como a seleção e utilização de modelos apropriados para o idioma.

### 1. Instalação de dependências
Execute o seguinte bloco para instalar e atualizar as dependências. Em seguida reinicie o kernel antes de executar os demais blocos.

In [1]:
%%capture
!pip install transformers datasets torch
!pip install sentencepiece
!pip install tf-keras
!pip install transformers[torch] -U
!pip install accelerate -U
!pip install transformers datasets torch accelerate -U

### 2. Carregamento do Corpus e Pré-processamento

In [24]:
import re
from datasets import load_dataset, Dataset

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Carrega o corpus do arquivo gerado pelo outro notebook deste repositório.
dataset = load_dataset('csv', data_files='combined_corpus.csv')

dataset = dataset.map(lambda x: {'texto': clean_text(x['texto'])})

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/356 [00:00<?, ? examples/s]

In [25]:
from datasets import DatasetDict

# Divide o dataset em 90% para treino e 10% para validação
train_test_split = dataset['train'].train_test_split(test_size=0.1)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test']
})

print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'texto', 'summary'],
        num_rows: 320
    })
    validation: Dataset({
        features: ['id', 'texto', 'summary'],
        num_rows: 36
    })
})


### 3. Escolha do Modelo Pré-Treinado

In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "rhaymison/flan-t5-portuguese-small-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

### 4. Tokenização e Preparação dos Dados

In [27]:
def tokenize_function(example):
    max_length = 512
    inputs = tokenizer(example['texto'], padding='max_length', truncation=True, max_length=max_length)
    labels = tokenizer(example['summary'], padding='max_length', truncation=True, max_length=max_length)

    inputs['labels'] = labels['input_ids']
    return inputs


tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=dataset['train'].column_names)



Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

### 5. Configuração do Treinamento

In [28]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)

### 6. Treinamento do Modelo

In [29]:
trainer.train()

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 4.199467658996582, 'eval_runtime': 46.807, 'eval_samples_per_second': 0.769, 'eval_steps_per_second': 0.107, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 2.454893112182617, 'eval_runtime': 60.8317, 'eval_samples_per_second': 0.592, 'eval_steps_per_second': 0.082, 'epoch': 2.0}


...

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 2.155787944793701, 'eval_runtime': 46.498, 'eval_samples_per_second': 0.774, 'eval_steps_per_second': 0.108, 'epoch': 4.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 2.1302297115325928, 'eval_runtime': 46.7757, 'eval_samples_per_second': 0.77, 'eval_steps_per_second': 0.107, 'epoch': 5.0}
{'train_runtime': 13563.9363, 'train_samples_per_second': 0.118, 'train_steps_per_second': 0.015, 'train_loss': 3.5478311157226563, 'epoch': 5.0}


TrainOutput(global_step=200, training_loss=3.5478311157226563, metrics={'train_runtime': 13563.9363, 'train_samples_per_second': 0.118, 'train_steps_per_second': 0.015, 'total_flos': 297424807526400.0, 'train_loss': 3.5478311157226563, 'epoch': 5.0})

### 7. Avaliação e Ajustes Finais

In [30]:
from pprint import pprint

results = trainer.evaluate()

pprint(results, indent=2)

  0%|          | 0/5 [00:00<?, ?it/s]

{ 'epoch': 5.0,
  'eval_loss': 2.1302297115325928,
  'eval_runtime': 44.6396,
  'eval_samples_per_second': 0.806,
  'eval_steps_per_second': 0.112}


### 8. Registrando o Modelo

In [31]:
model.save_pretrained('./final_model')
tokenizer.save_pretrained('./final_model')

('./final_model\\tokenizer_config.json',
 './final_model\\special_tokens_map.json',
 './final_model\\tokenizer.json')

### 9. Uso do Modelo

In [32]:
from transformers import pipeline

# Carrega o modelo treinado
summarizer = pipeline('summarization', model='./final_model', tokenizer='./final_model')

# Caminho para o arquivo de teste
test_file_path = 'input-test.txt'

# Lê todo o conteúdo do arquivo de teste
with open(test_file_path, 'r', encoding='utf-8') as file:
    test_text = file.read().strip()

# Define o limite máximo de tokens na saída
max_length_output = 1024 
min_length_output = 100

# Resume o texto do arquivo
summary = summarizer("sumarize:" + test_text, max_length=max_length_output, min_length=min_length_output, do_sample=False)[0]['summary_text']

# Exibir o resumo
print("Resumo do texto:")
print(summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (5658 > 512). Running this sequence through the model will result in indexing errors


Resumo do texto:
Setor de Defesa é aprovado pela Resoluço ConSUG/MD no 6, de 26 de dezembro de 2018, e apresenta o Plano Estratégico Setorial 2024-2027, com os objetivos de recursos para o desenvolvimento nacional, a integrar a manobra, o engajamento, integridade nacionais, atividades e inovaçes, em diversas áreas de responsabilidade do Estado brasileiro. O Plano Plurianual 2024/2027 tem o preparamento de estudos, incluindo inteligência, vigilância, meteorologia e oceanografia, ampliar o conhecimento dos medidas e facilitar esforço da cidadania


#### Teste 1
Parâmetros:
```py
model_name = "rhaymison/flan-t5-portuguese-small-summarization"
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True
)
```
Resultado:

Setor de Defesa é aprovado pela Resoluço ConSUG/MD no 6, de 11 de dezembro de 2020, e especialistas apontam que o Plano Estratégico Setorial 2024-2027 está alinhado com a cadeia de valor setorial, adequadamente preparadas e fortalecimento dos objetivos nacionais, como a primeira esforço da cidadania. Resultado de estudos e debates sobre temas ligados ao enfrentamento nacional e defeso no significa exclusividade para o desenvolvimento


#### Teste 2
Parâmetros:
```py
model_name = "rhaymison/flan-t5-portuguese-small-summarization"
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True
)
```

Resultado:

Conheça o Plano Estratégico Setorial 2024-2027 com o PESD 2020-2027, aprovado pela Resoluço ConSUG/MD no 6, de 11 de dezembro de 2019, apresenta a identidade estratégica do Setor de Defesa, como se segue em vigor, e esforço nacional, na garantia da lei e da ordem; entenda os objetivos para o desenvolvimento de recursos. Governo tem ainda especialistas que aconteceram o plano após a primeira açes de efetividade ao sistema

#### Teste 3

Parâmetros:
```py
model_name = "rhaymison/flan-t5-portuguese-small-summarization"
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True
)
```

Resultado:

O Plano Estratégico Setorial 2024-2027 é aprovado pela Resoluço ConSUG/MD no 6, de 11 de dezembro de 2019, e o Plano estabelece a prioridade do Setor de Defesa para o desenvolvimento nacional, com os objetivos de serem empregados para a integraçes nacionais, o controle, a proteça, em apoio à poltica externa, para promover a confiança mtua, coeso ambiental e controles martimas. Os efeitos ainda no acontece

### Teste t5-portuguese-small-summarization

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Carrega o tokenizer e o modelo que foram treinados juntos
tokenizer = AutoTokenizer.from_pretrained("rhaymison/flan-t5-portuguese-small-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained("rhaymison/flan-t5-portuguese-small-summarization")

# Configura o pipeline de sumarização
summarization = pipeline("summarization", model=model, tokenizer=tokenizer)

# Texto de entrada
test_file_path = 'input-test.txt'
with open(test_file_path, 'r', encoding='utf-8') as file:
    test_text = file.read().strip()

# Realiza a sumarização
output = summarization("sumarize: " + test_text)

# Exibe o resumo
print("Resumo do texto:")
print(output[0]['summary_text'])


Token indices sequence length is longer than the specified maximum sequence length for this model (5657 > 512). Running this sequence through the model will result in indexing errors


Resumo do texto:
Setor de Defesa é aprovado pela Resoluço ConSUG/MD no 2, de 2019, após açes em vigor com os objetivos de recursos. Plano estratégico deve ser realizado pelo PES 2024-2027, e especialistas no so preparados para o desenvolvimento nacional, ainda como a esforço da cidadania.


**Resultado sem Tuning**

Setor de Defesa é aprovado pela Resoluço ConSUG/MD no 2, de 2019, após açes em vigor com os objetivos de recursos. Plano estratégico deve ser realizado pelo PES 2024-2027, e especialistas no so preparados para o desenvolvimento nacional, ainda como a esforço da cidadania.

**Resultado Final com Tuning**

Setor de Defesa é aprovado pela Resoluço ConSUG/MD no 6, de 26 de dezembro de 2018, e apresenta o Plano Estratégico Setorial 2024-2027, com os objetivos de recursos para o desenvolvimento nacional, a integrar a manobra, o engajamento, integridade nacionais, atividades e inovaçes, em diversas áreas de responsabilidade do Estado brasileiro. O Plano Plurianual 2024/2027 tem o preparamento de estudos, incluindo inteligência, vigilância, meteorologia e oceanografia, ampliar o conhecimento dos medidas e facilitar esforço da cidadania

**Resumo feito pelo chatGPT 4o**

O Ministério da Defesa aprovou o Plano Estratégico Setorial 2024-2027 (PES 2024-2027), que estabelece prioridades para a implementação do Planejamento Estratégico Setorial de Defesa 2020-2031. O plano visa alinhar o planejamento estratégico com o orçamentário, priorizando objetivos e iniciativas estratégicas de defesa (OSD e IED), como o desenvolvimento de capacidades militares, apoio à política externa, desenvolvimento científico e tecnológico, e fortalecimento dos setores nuclear, cibernético e espacial. As metas serão acompanhadas anualmente pelo Conselho Superior de Governança do Ministério da Defesa, que também gerenciará riscos estratégicos relacionados a restrições orçamentárias e embargos tecnológicos.